In [3]:
from swebench import MAP_VERSION_TO_INSTALL, get_eval_refs, get_instances

In [4]:
dataset = "princeton-nlp/SWE-bench_Verified"
task_instances = get_eval_refs(data_path_or_name=dataset)
print(len(task_instances))


500


In [5]:
from collections import Counter
Counter([x.split("-")[0] for x in task_instances.keys()])

Counter({'django__django': 231,
         'sympy__sympy': 75,
         'sphinx': 44,
         'matplotlib__matplotlib': 34,
         'scikit': 32,
         'astropy__astropy': 22,
         'pydata__xarray': 22,
         'pytest': 19,
         'pylint': 10,
         'psf__requests': 8,
         'mwaskom__seaborn': 2,
         'pallets__flask': 1})

In [31]:
dataset_save_dir = f"../../temp/{dataset.split('/')[-1]}"

In [32]:
instance_key = next(iter(task_instances))

In [33]:
instance = task_instances[instance_key]

In [34]:
import os
import json
from pathlib import Path

In [35]:

def load_json(path: Path):
    with open(path, "r") as f:
        return json.load(f)

def save_json(data, path: Path):
    with open(path, "w") as f:
        json.dump(data, f, indent=4)


In [51]:
for instance_key in task_instances:
    Path(dataset_save_dir, instance_key).mkdir(parents=True, exist_ok=True)
    instance = task_instances[instance_key]
    metadata = {
        "repo": instance['repo'],
        'instance_id': instance['instance_id'],
        'base_commit': instance['base_commit'],
        'created_at': instance['created_at'],
        'version': instance['version'],
    }
    with open(Path(dataset_save_dir, instance_key, "patch.txt"), 'w') as handle:
        handle.write(instance['patch'])
    with open(Path(dataset_save_dir, instance_key, "test_patch.txt"), 'w') as handle:
        handle.write(instance['test_patch'])
    with open(Path(dataset_save_dir, instance_key, "problem_statement.txt"), 'w') as handle:
        handle.write(instance['problem_statement'])
    with open(Path(dataset_save_dir, instance_key, "hints_text.txt"), 'w') as handle:
        handle.write(instance['hints_text'])
    
    save_json(metadata, Path(dataset_save_dir, instance_key, "metadata.json"))
    

In [67]:
import random

In [74]:
test_instances = random.sample(sorted(task_instances), 25)
test_instances = ['sympy__sympy-24539',
 'django__django-16493',
 'sphinx-doc__sphinx-8595',
 'django__django-14373',
 'sympy__sympy-22914',
 'django__django-16255',
 'sympy__sympy-16766',
 'sympy__sympy-20801',
 'django__django-13590',
 'django__django-7530',
 'scikit-learn__scikit-learn-11578',
 'django__django-12143',
 'pydata__xarray-4966',
 'django__django-11603',
 'django__django-16612',
 'django__django-13741',
 'django__django-14089',
 'django__django-10880',
 'django__django-15467',
 'django__django-14500',
 'django__django-13810',
 'sympy__sympy-21847',
 'scikit-learn__scikit-learn-10297',
 'django__django-13820',
 'sympy__sympy-24066']

In [75]:
import shutil

In [76]:
for file_name in test_instances:
    source_path = os.path.join("../../temp/SWE-bench_Verified", file_name)
    destination_path = os.path.join("../../temp/test", file_name)
    shutil.copytree(source_path, destination_path)

In [77]:
for file_name in os.listdir("../../temp/SWE-bench_Verified"):
    if file_name in test_instances: continue
    source_path = os.path.join("../../temp/SWE-bench_Verified", file_name)
    destination_path = os.path.join("../../temp/train", file_name)
    shutil.copytree(source_path, destination_path)

In [9]:
import glob
for files in glob.glob("../generated/sphinx-doc__sphinx/*/Dockerfile"):
    content = open(files).read()
    content = "\n".join([line for line in content.splitlines() if "sed" not in line])
    open(files, "w").write(content)
    

In [8]:
print("\n".join([line for line in content.splitlines() if "sed" not in line]))

FROM composio/swe:py3.9

# Switch user
USER user

# Go to user dir
WORKDIR /home/user

# Clone github repository
RUN git config --global http.postBuffer 157286400 && git config --global --add safe.directory /home/user/sphinx && git clone --depth 1 https://github.com/sphinx-doc/sphinx

# Set repository as workdir
WORKDIR /home/user/sphinx

# Fetch the base commit
RUN git fetch --depth 1 origin 4b452338f914d4f6b54704222d70ae8a746e3db5

# Checkout to base commit
RUN git checkout 4b452338f914d4f6b54704222d70ae8a746e3db5

WORKDIR /home/user

RUN mkdir -p /home/user/.composio/tmp
COPY deeplake /home/user/.composio/tmp/sphinx/deeplake
COPY fqdn_cache.json /home/user/.composio/tmp/sphinx

# Install dependecies
RUN /home/user/.dev/bin/python -m pip install "tox" || echo "$?"

# Pre install

# Install package
RUN /home/user/.dev/bin/python -m pip install -e .[test] || echo "$?"

ENV HOME=/home/user/

WORKDIR /home/user/sphinx

# Switch to root
USER root


In [ ]:
""